## Learning Classic ConvNet architectures (and associated layers) as per DLS course of Andrew Ng

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models

In [31]:
x = np.random.randn(10, 10)

# Add batch dimension
x_expanded = np.expand_dims(x, axis=0)

# Define Flatten layer
flatten_layer = layers.Flatten()

# Apply flattening
flattened = flatten_layer(x_expanded)

# Print results
print("Original shape:", x.shape)
print("After expanding:", x_expanded.shape)
print("Flattened shape:", flattened.shape)
# print("Flattened output:", flattened.numpy())

Original shape: (10, 10)
After expanding: (1, 10, 10)
Flattened shape: (1, 100)


### Le-net 5

In [74]:
# Define layers of LeNet-5 as a list
lenet_5_layers = [
    layers.Input(shape=(32, 32, 1)),
    layers.Conv2D(6, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=2),
    layers.Conv2D(16, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=2),
    layers.Conv2D(120, kernel_size=5, activation='tanh'),
    layers.Flatten(),
    layers.Dense(84, activation='tanh'),
    layers.Dense(10, activation='softmax')
]

model = models.Sequential(lenet_5_layers,name="LeNet-5")
model.summary()

Model: "LeNet-5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_17            │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_18            │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

Step through the individual layers to understand better

In [80]:
# Input batch (single 32x32 grayscale image) - note the 1 added in first dimension which is the batch dimension
input_batch = np.random.rand(1, 32, 32, 1).astype(np.float32)

# Manual forward pass
layer_outputs = [None] * len(lenet_5_layers)
layer_outputs[0] = input_batch

print(f"Layer 0: Input Layer | Output Shape: {layer_outputs[0].shape} | Nodes: -")

for l in range(1, len(lenet_5_layers)):
    layer_outputs[l] = lenet_5_layers[l](layer_outputs[l - 1])
    
    # Determine "nodes"
    layer = lenet_5_layers[l]
    shape = layer_outputs[l].shape
    
    if isinstance(layer, layers.Conv2D):
        nodes = layer.filters
    elif isinstance(layer, layers.Dense):
        nodes = layer.units
    elif isinstance(layer, layers.Flatten):
        nodes = shape[-1]
    elif isinstance(layer, layers.AveragePooling2D):
        shape = layer_outputs[l].shape
        nodes = shape[1] * shape[2] * shape[3]
    else:
        nodes = "-"

    print(f"Layer {l}: {layer.__class__.__name__:<20} | Output Shape: {shape} | Nodes: {nodes}")

Layer 0: Input Layer | Output Shape: (1, 32, 32, 1) | Nodes: -
Layer 1: Conv2D               | Output Shape: (1, 28, 28, 6) | Nodes: 6
Layer 2: AveragePooling2D     | Output Shape: (1, 14, 14, 6) | Nodes: 1176
Layer 3: Conv2D               | Output Shape: (1, 10, 10, 16) | Nodes: 16
Layer 4: AveragePooling2D     | Output Shape: (1, 5, 5, 16) | Nodes: 400
Layer 5: Conv2D               | Output Shape: (1, 1, 1, 120) | Nodes: 120
Layer 6: Flatten              | Output Shape: (1, 120) | Nodes: 120
Layer 7: Dense                | Output Shape: (1, 84) | Nodes: 84
Layer 8: Dense                | Output Shape: (1, 10) | Nodes: 10


### AlexNet

In [6]:
# Define AlexNet-style architecture
alexnet_layers = [
    layers.Input(shape=(227, 227, 1)),                           # Layer 0
    layers.Conv2D(96, kernel_size=11, strides=4, activation='relu'),   # Layer 1
    layers.MaxPooling2D(pool_size=3, strides=2),                        # Layer 2
    layers.Conv2D(256, kernel_size=5, padding='same', activation='relu'),  # Layer 3
    layers.MaxPooling2D(pool_size=3, strides=2),                        # Layer 4
    layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),  # Layer 5
    layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),  # Layer 6
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),  # Layer 7
    layers.MaxPooling2D(pool_size=3, strides=2),                        # Layer 8
    layers.Flatten(),                                                  # Layer 9
    layers.Dense(4096, activation='relu'),                             # Layer 10
    layers.Dense(4096, activation='relu'),                             # Layer 11
    layers.Dense(1000, activation='softmax')                           # Layer 12
]
model = models.Sequential(alexnet_layers,name="AlexNet")
model.summary()

Model: "AlexNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 55, 55, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,355,112 (237.87 MB)

 Trainable params: 62,355,112 (237.87 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Simulated grayscale image (1 sample)
input_batch = np.random.rand(1, 227, 227, 1).astype(np.float16)

# Manual forward pass with output shape and node count
layer_outputs = [None] * len(alexnet_layers)
layer_outputs[0] = input_batch

print(f"Layer 0: Input Layer          | Output Shape: {layer_outputs[0].shape} | Nodes: -")

for l in range(1, len(alexnet_layers)):
    layer_outputs[l] = alexnet_layers[l](layer_outputs[l - 1])
    shape = layer_outputs[l].shape
    layer = alexnet_layers[l]

    if isinstance(layer, layers.Conv2D):
        nodes = shape[1] * shape[2] * shape[3]
    elif isinstance(layer, layers.Dense):
        nodes = layer.units
    elif isinstance(layer, layers.Flatten):
        nodes = shape[-1]
    elif isinstance(layer, layers.MaxPooling2D):
        nodes = shape[1] * shape[2] * shape[3]
    else:
        nodes = "-"

    print(f"Layer {l:2}: {layer.__class__.__name__:<20} | Output Shape: {shape} | Nodes: {nodes}")


Layer 0: Input Layer          | Output Shape: (1, 227, 227, 1) | Nodes: -
Layer  1: Conv2D               | Output Shape: (1, 55, 55, 96) | Nodes: 290400
Layer  2: MaxPooling2D         | Output Shape: (1, 27, 27, 96) | Nodes: 69984
Layer  3: Conv2D               | Output Shape: (1, 27, 27, 256) | Nodes: 186624
Layer  4: MaxPooling2D         | Output Shape: (1, 13, 13, 256) | Nodes: 43264
Layer  5: Conv2D               | Output Shape: (1, 13, 13, 384) | Nodes: 64896
Layer  6: Conv2D               | Output Shape: (1, 13, 13, 384) | Nodes: 64896
Layer  7: Conv2D               | Output Shape: (1, 13, 13, 256) | Nodes: 43264
Layer  8: MaxPooling2D         | Output Shape: (1, 6, 6, 256) | Nodes: 9216
Layer  9: Flatten              | Output Shape: (1, 9216) | Nodes: 9216
Layer 10: Dense                | Output Shape: (1, 4096) | Nodes: 4096
Layer 11: Dense                | Output Shape: (1, 4096) | Nodes: 4096
Layer 12: Dense                | Output Shape: (1, 1000) | Nodes: 1000


### VGG-16

In [8]:
# VGG-16 layer sequence (simplified for grayscale: input shape (224, 224, 1))
vgg16_layers = [
    layers.Input(shape=(224, 224, 1)),                             # Layer 0
    layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),  # Layer 1
    layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'),  # Layer 2
    layers.MaxPooling2D(pool_size=2, strides=2),                   # Layer 3

    layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'), # Layer 4
    layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'), # Layer 5
    layers.MaxPooling2D(pool_size=2, strides=2),                   # Layer 6

    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'), # Layer 7
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'), # Layer 8
    layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'), # Layer 9
    layers.MaxPooling2D(pool_size=2, strides=2),                   # Layer 10

    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 11
    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 12
    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 13
    layers.MaxPooling2D(pool_size=2, strides=2),                   # Layer 14

    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 15
    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 16
    layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'), # Layer 17
    layers.MaxPooling2D(pool_size=2, strides=2),                   # Layer 18

    layers.Flatten(),                                              # Layer 19
    layers.Dense(4096, activation='relu'),                         # Layer 20
    layers.Dense(4096, activation='relu'),                         # Layer 21
    layers.Dense(1000, activation='softmax')                       # Layer 22
]
model = models.Sequential(vgg16_layers,name="VGG-16")
model.summary()

Model: "VGG-16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)              │ (None, 224, 224, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,356,392 (527.79 MB)

 Trainable params: 138,356,392 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Simulated grayscale input image
input_batch = np.random.rand(1, 224, 224, 1).astype(np.float16)

# Manual forward pass
layer_outputs = [None] * len(vgg16_layers)
layer_outputs[0] = input_batch

print(f"Layer  0: Input Layer           | Output Shape: {layer_outputs[0].shape} | Nodes: -")

for l in range(1, len(vgg16_layers)):
    layer_outputs[l] = vgg16_layers[l](layer_outputs[l - 1])
    shape = layer_outputs[l].shape
    layer = vgg16_layers[l]

    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.MaxPooling2D):
        nodes = shape[1] * shape[2] * shape[3]
    elif isinstance(layer, layers.Flatten):
        nodes = shape[-1]
    elif isinstance(layer, layers.Dense):
        nodes = layer.units
    else:
        nodes = "-"

    print(f"Layer {l:2}: {layer.__class__.__name__:<20} | Output Shape: {shape} | Nodes: {nodes}")

Layer  0: Input Layer           | Output Shape: (1, 224, 224, 1) | Nodes: -
Layer  1: Conv2D               | Output Shape: (1, 224, 224, 64) | Nodes: 3211264
Layer  2: Conv2D               | Output Shape: (1, 224, 224, 64) | Nodes: 3211264
Layer  3: MaxPooling2D         | Output Shape: (1, 112, 112, 64) | Nodes: 802816
Layer  4: Conv2D               | Output Shape: (1, 112, 112, 128) | Nodes: 1605632
Layer  5: Conv2D               | Output Shape: (1, 112, 112, 128) | Nodes: 1605632
Layer  6: MaxPooling2D         | Output Shape: (1, 56, 56, 128) | Nodes: 401408
Layer  7: Conv2D               | Output Shape: (1, 56, 56, 256) | Nodes: 802816
Layer  8: Conv2D               | Output Shape: (1, 56, 56, 256) | Nodes: 802816
Layer  9: Conv2D               | Output Shape: (1, 56, 56, 256) | Nodes: 802816
Layer 10: MaxPooling2D         | Output Shape: (1, 28, 28, 256) | Nodes: 200704
Layer 11: Conv2D               | Output Shape: (1, 28, 28, 512) | Nodes: 401408
Layer 12: Conv2D               | 